In [1]:
# pip install databricks-sdk openai
# !pip install databricks-sql-connector
# !pip install --upgrade numpy pandas
# !pip install pandas==2.2.2
# !pip install -U langchain-openai
# !pip install numpy<2
# !pip install pyarrow


  Attempting uninstall: langchain-core

    Found existing installation: langchain-core 0.3.65

    Uninstalling langchain-core-0.3.65:

   ---------------------------------------- 0/2 [langchain-core]
      Successfully uninstalled langchain-core-0.3.65
   ---------------------------------------- 0/2 [langchain-core]
   ---------------------------------------- 0/2 [langchain-core]
   ---------------------------------------- 0/2 [langchain-core]
   ---------------------------------------- 0/2 [langchain-core]
   ---------------------------------------- 0/2 [langchain-core]
   ---------------------------------------- 0/2 [langchain-core]
   ---------------------------------------- 0/2 [langchain-core]
   ---------------------------------------- 0/2 [langchain-core]
   ---------------------------------------- 0/2 [langchain-core]
   ---------------------------------------- 0/2 [langchain-core]
   ---------------------------------------- 0/2 [langchain-core]
  Attempting uninstall: langc

The system cannot find the file specified.


In [2]:
# === 1. Imports & Config ===
import os
import pandas as pd
from databricks import sql
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage
from dotenv import load_dotenv

load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")

# Databricks config
DATABRICKS_SERVER_HOSTNAME = os.getenv("DATABRICKS_SERVER_HOSTNAME")
DATABRICKS_ACCESS_TOKEN = os.getenv("DATABRICKS_ACCESS_TOKEN")
WAREHOUSE_ID = os.getenv("WAREHOUSE_ID")

CATALOG = "main"
SCHEMA = "bi_schema"
TABLE = "orders_superstore"

# LLM setup
#llm = ChatOpenAI(model_name="gpt-4", temperature=0)
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# === 2. Functions ===

def load_table_from_databricks(catalog, schema, table):
    connection = sql.connect(
        server_hostname=DATABRICKS_SERVER_HOSTNAME,
        http_path=f"/sql/1.0/warehouses/{WAREHOUSE_ID}",
        access_token=DATABRICKS_ACCESS_TOKEN
    )
    query = f"SELECT * FROM {catalog}.{schema}.{table}"
    cursor = connection.cursor()
    cursor.execute(query)
    columns = [desc[0] for desc in cursor.description]
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=columns)
    cursor.close()
    connection.close()
    return df

def analyze_internal_insights(df: pd.DataFrame, user_prompt: str, table_name: str = "Orders"):
    # Step 1: Extract category values
    all_categories = df["Category"].dropna().unique().tolist()
    all_subcategories = df["Sub-Category"].dropna().unique().tolist()
   
    # Step 2: Use LLM to map prompt to subcategories
    category_prompt = ChatPromptTemplate.from_template("""
You are helping analyze a sales dataset. The dataset includes:

Main Categories: {categories}
Sub-Categories: {subcategories}

Given this campaign prompt:
"{user_prompt}"

Return a dedupped Python list of relevant Sub-Categories that match the prompt.
Example: ["Binders", "Art", "Appliances"]
""")

    messages = category_prompt.format_messages(
        categories=all_categories,
        subcategories=all_subcategories,
        user_prompt=user_prompt
    )
    response = llm.invoke(messages).content

    try:
        matched_subcategories = eval(response)
    except:
        return "LLM response could not be parsed. Response was:\n" + response, None, None, None

    if not matched_subcategories:
        return "No relevant sub-categories found by the LLM.", None, None, None

    # Step 3: Filter data
    df_filtered = df[df["Sub-Category"].isin(matched_subcategories)].copy()
    df_filtered["Order_Date"] = pd.to_datetime(df_filtered["Order_Date"])
    df_filtered["year"] = df_filtered["Order_Date"].dt.year

    # Step 4: Identify the latest two years in the dataset
    latest_years = sorted(df_filtered["year"].dropna().unique())[-2:]
    if len(latest_years) < 2:
        raise ValueError("Not enough years of data for YoY comparison.")

    year_new, year_old = latest_years[1], latest_years[0]

    # Step 5: YoY aggregation
    grouped = df_filtered.groupby(["Sub-Category", "Region", "year"]).agg({
        "Sales": "sum",
        "Profit": "sum"
    }).reset_index()

    df_new = grouped[grouped["year"] == year_new].set_index(["Sub-Category", "Region"])
    df_old = grouped[grouped["year"] == year_old].set_index(["Sub-Category", "Region"])

    yoy = df_new.join(df_old, lsuffix=f"_{year_new}", rsuffix=f"_{year_old}", how="inner")
    yoy["sales_yoy"] = ((yoy[f"Sales_{year_new}"] - yoy[f"Sales_{year_old}"]) / yoy[f"Sales_{year_old}"]) * 100
    yoy["profit_yoy"] = ((yoy[f"Profit_{year_new}"] - yoy[f"Profit_{year_old}"]) / yoy[f"Profit_{year_old}"]) * 100
    yoy.reset_index(inplace=True)


    # Step 6: SQL generation
    sql_list = ", ".join(f"'{s}'" for s in matched_subcategories)
    sql_script = f"""
SELECT 
    `Sub-Category`,
    `Region`,
    YEAR(`Order_Date`) AS year,
    SUM(Sales) AS total_sales,
    SUM(Profit) AS total_profit
FROM {table_name}
WHERE `Sub-Category` IN ({sql_list})
GROUP BY `Sub-Category`, `Region`, YEAR(`Order_Date`)
""".strip()

    # Step 7: Insights summary
    summary_prompt = ChatPromptTemplate.from_template("""
You are a BI analyst. Summarize the following YoY data in clear bullet points.
Focus on major % increases or decreases in sales or profit by sub-category and region.

Data:
{yoy_data}
""")
    summary_input = summary_prompt.format_messages(yoy_data=yoy.to_csv(index=False))
    insight_summary = llm.invoke(summary_input).content

    return df_filtered, yoy, sql_script, insight_summary#, matched_subcategories

web_summary = [
    "Target is promoting bundle offers in Northeast U.S.",
    "Amazon is running 15% discounts on school supplies nationwide.",
    "Back-to-school themed influencer campaigns are trending on TikTok."
]

def generate_campaign_report(web_summaries,insight_summary, user_topic):
    prompt = f"""
You're a marketing analyst. Create a one-page summary report combining web insights and sales data insight summary for a {user_topic}.
Focus on trends, regional patterns, and recommended actions.

Web insights:
{chr(10).join(f"- {ws}" for ws in web_summaries)}

Sales data summaries:
{chr(10).join(f"- {cs}" for cs in insight_summary)}

Return output as:
1. Executive Summary
2. Key Insights
3. Recommended Actions
"""
    messages = [HumanMessage(content=prompt)]
    response = llm.invoke(messages)
    return response.content

# # === 3. Execution Flow ===
if __name__ == "__main__":
    user_topic = input("📌 Enter campaign topic: ")
    print("⏳ Step 1: Loading data from Databricks...")
    df = load_table_from_databricks(CATALOG, SCHEMA, TABLE)
    print(df["Order_Date"].max())


    print("⚙️ Step 2: Analyzing internal insights...")
    df_filtered, yoy, sql_script, insight_summary = analyze_internal_insights(df, user_topic)
    print(sql_script)
    #print(yoy)
    print(insight_summary)

    if insight_summary:
        print("🧠 Generating campaign report...")
        report = generate_campaign_report(web_summary,insight_summary, user_topic)
        print(report)
    else:
        print("❌ No insights generated. Please refine your prompt.")


📌 Enter campaign topic: Plan a Back-to-School campaign. Focus on writing tools, and classroom furniture
⏳ Step 1: Loading data from Databricks...
2024-12-30 00:00:00
⚙️ Step 2: Analyzing internal insights...
SELECT 
    `Sub-Category`,
    `Region`,
    YEAR(`Order_Date`) AS year,
    SUM(Sales) AS total_sales,
    SUM(Profit) AS total_profit
FROM Orders
WHERE `Sub-Category` IN ('Binders', 'Paper', 'Labels', 'Art', 'Chairs', 'Tables')
GROUP BY `Sub-Category`, `Region`, YEAR(`Order_Date`)
- Art sub-category in the Central region saw a significant increase in both sales and profit YoY, with sales increasing by 50.75% and profit by 90.44%.
- Binders sub-category in the South region experienced a substantial sales increase of 170.98% YoY, but a decrease in profit by 34.31%.
- Chairs sub-category in the South region had a notable increase in sales by 147.29% and profit by 51.33% YoY.
- Labels sub-category in the South region also showed a significant increase in both sales and profit YoY, w

In [ ]:
user_prompt = "Plan a Back-to-School campaign. Focus on binders, tables, writing tools, and classroom furniture."